In [0]:
# Configuration 
eventhubs_connection_string = "YOUR_KEY_HERE"

eventhubs_name = "crypto-stream"

# For testing - we'll use sample data first
use_sample_data = True

## Sample data generator for testing

In [0]:

import json
import random
import time
from datetime import datetime

def generate_sample_crypto_data():
    symbols = ['BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'DOGEUSDT', 'SOLUSDT']
    exchanges = ['binance', 'coinbase', 'kraken']
     
    data = {
        'exchange': random.choice(exchanges),
        'symbol': random.choice(symbols),
        'price': round(random.uniform(100, 50000), 2),
        'volume': round(random.uniform(1000, 1000000), 2),
        'price_change': round(random.uniform(-1000, 1000), 2),
        'timestamp': datetime.now().timestamp() * 1000
    }
    return data

# Test it
print("Sample data:", generate_sample_crypto_data())


Sample data: {'exchange': 'coinbase', 'symbol': 'BTCUSDT', 'price': 42505.72, 'volume': 830744.8, 'price_change': -720.59, 'timestamp': 1756437923314.821}


## Configuration

In [0]:
# Configuration
binance_api_url = "https://api.binance.us/api/v3/ticker/24hr"
target_symbols = ['BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'DOGEUSDT', 'SOLUSDT', 'XRPUSDT', 'DOTUSDT']

# Import libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time
from IPython.display import display, clear_output
import numpy as np

# Set professional styling
plt.style.use('default')
plt.rcParams['figure.facecolor'] = '#f8f9fa'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['grid.color'] = '#e9ecef'
plt.rcParams['text.color'] = '#2c3e50'
plt.rcParams['axes.labelcolor'] = '#2c3e50'
plt.rcParams['xtick.color'] = '#2c3e50'
plt.rcParams['ytick.color'] = '#2c3e50'

print("Configuration and libraries loaded successfully")

Configuration and libraries loaded successfully


## Data Fetcher Function

In [0]:
def fetch_binance_data():
    """Fetch real-time data from Binance API"""
    try:
        response = requests.get(binance_api_url, timeout=10)
        
        if response.status_code != 200:
            print(f"API returned status: {response.status_code}")
            return []
            
        all_data = response.json()
        processed_data = []
        
        for item in all_data:
            if item['symbol'] in target_symbols:
                price_change = float(item['priceChangePercent'])
                processed_data.append({
                    'symbol': item['symbol'],
                    'price': float(item['lastPrice']),
                    'volume': float(item['volume']),
                    'price_change_percent': price_change,
                    'high_price': float(item['highPrice']),
                    'low_price': float(item['lowPrice']),
                    'quote_volume': float(item['quoteVolume']),
                    'timestamp': datetime.now().strftime('%H:%M:%S'),
                    'is_positive': price_change >= 0
                })
        
        # Sort by volume (market activity)
        processed_data.sort(key=lambda x: x['volume'], reverse=True)
        return processed_data
        
    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

    finally:
        clear_output(wait=True)

# Test the data fetcher
test_data = fetch_binance_data()
print(f"Fetched {len(test_data)} records")
if test_data:
    for item in test_data[:3]:  # Show first 3
        print(f"{item['symbol']}: ${item['price']:,.2f} ({item['price_change_percent']:+.2f}%)")

Fetched 7 records
DOGEUSDT: $0.22 (+0.40%)
XRPUSDT: $2.94 (-1.17%)
ADAUSDT: $0.85 (-0.92%)


## Dashboard Graphs

In [0]:
def create_professional_dashboard(data):
    """Create a clean, professional dashboard without emojis"""
    if not data:
        print("No data available for dashboard")
        return
    
    clear_output(wait=True)
    
    # Create figure with subplots
    fig = plt.figure(figsize=(16, 12))
    fig.suptitle('Real-Time Cryptocurrency Analytics Dashboard', 
                 fontsize=16, fontweight='bold', y=0.98)
    
    # Create grid layout
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    ax1 = fig.add_subplot(gs[0, 0])  # Prices
    ax2 = fig.add_subplot(gs[0, 1])  # Price changes
    ax3 = fig.add_subplot(gs[1, 0])  # Volume
    ax4 = fig.add_subplot(gs[1, 1])  # Price ranges
    ax5 = fig.add_subplot(gs[2, 0])  # Market sentiment
    ax6 = fig.add_subplot(gs[2, 1])  # Performance table
    
    # Extract data
    symbols = [item['symbol'] for item in data]
    prices = [item['price'] for item in data]
    changes = [item['price_change_percent'] for item in data]
    volumes = [item['volume'] for item in data]
    
    # Color scheme
    positive_color = '#2ecc71'  # Green
    negative_color = '#e74c3c'  # Red
    neutral_color = '#3498db'   # Blue
    colors = [positive_color if ch >= 0 else negative_color for ch in changes]
    
    # 1. CURRENT PRICES
    bars = ax1.bar(symbols, prices, color=colors, alpha=0.8, edgecolor='white', linewidth=1)
    ax1.set_title('Current Prices (USDT)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Price', fontweight='bold')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3)
    
    # Add price labels
    for bar, price in zip(bars, prices):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 1.01, 
                f'${price:,.0f}', ha='center', va='bottom', 
                fontweight='bold', fontsize=9)
    
    # 2. PRICE CHANGES (24h)
    ax2.barh(symbols, changes, color=colors, alpha=0.8, edgecolor='white', linewidth=1)
    ax2.set_title('24-Hour Price Change (%)', fontsize=12, fontweight='bold')
    ax2.axvline(x=0, color='black', linestyle='--', alpha=0.7)
    ax2.set_xlabel('Percentage Change', fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Add percentage labels
    for i, (change, symbol) in enumerate(zip(changes, symbols)):
        ax2.text(change + (0.3 if change >= 0 else -0.3), i, 
                f'{change:+.1f}%', ha='left' if change >= 0 else 'right', 
                va='center', fontweight='bold', fontsize=9)
    
    # 3. TRADING VOLUME
    ax3.bar(symbols, volumes, color=neutral_color, alpha=0.8, edgecolor='white', linewidth=1)
    ax3.set_title('24-Hour Trading Volume', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Volume', fontweight='bold')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(True, alpha=0.3)
    ax3.ticklabel_format(style='scientific', axis='y', scilimits=(6,6))
    
    # 4. PRICE RANGES
    for i, item in enumerate(data):
        # Plot high-low range
        ax4.plot([i, i], [item['low_price'], item['high_price']], 
                color='#34495e', linewidth=3, alpha=0.7, marker='|', markersize=8)
        # Plot current price
        ax4.plot(i, item['price'], 'o', color=positive_color if item['is_positive'] else negative_color, 
                markersize=8, label='Current Price' if i == 0 else "")
    
    ax4.set_title('Price Ranges (High - Low)', fontsize=12, fontweight='bold')
    ax4.set_xticks(range(len(symbols)))
    ax4.set_xticklabels(symbols, rotation=45)
    ax4.set_ylabel('Price (USDT)', fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. MARKET SENTIMENT
    positive_count = sum(1 for ch in changes if ch >= 0)
    negative_count = len(changes) - positive_count
    sentiment_labels = ['Positive', 'Negative']
    sentiment_sizes = [positive_count, negative_count]
    sentiment_colors = [positive_color, negative_color]
    
    wedges, texts, autotexts = ax5.pie(sentiment_sizes, labels=sentiment_labels, colors=sentiment_colors,
                                      autopct='%1.0f%%', startangle=90, textprops={'fontweight': 'bold'})
    ax5.set_title('Market Sentiment', fontsize=12, fontweight='bold')
    
    # 6. PERFORMANCE TABLE
    table_data = []
    for item in data:
        change_color = positive_color if item['price_change_percent'] >= 0 else negative_color
        table_data.append([
            item['symbol'],
            f"${item['price']:,.2f}",
            f"{item['price_change_percent']:+.2f}%",
            f"{item['volume']:,.0f}"
        ])
    
    ax6.axis('off')
    table = ax6.table(cellText=table_data,
                     colLabels=['Symbol', 'Price', 'Change%', 'Volume'],
                     cellLoc='center',
                     loc='center',
                     colColours=['#34495e'] * 4,
                     cellColours=[['#f8f9fa'] * 4] * len(data))
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    # Set text colors for positive/negative changes
    for i in range(1, len(table_data) + 1):
        change_value = float(table_data[i-1][2].replace('%', '').replace('+', '').replace('-', ''))
        if '+' in table_data[i-1][2]:
            table[(i, 2)].set_text_props(color=positive_color, weight='bold')
        else:
            table[(i, 2)].set_text_props(color=negative_color, weight='bold')
    
    # Add timestamp and info
    plt.figtext(0.02, 0.02, 
               f'Last Updated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")} | '
               f'Data Source: Binance API | '
               f'Tracking: {len(data)} cryptocurrencies',
               fontsize=9, style='italic', color='#7f8c8d')
    
    # Add explanation
    explanation_text = (
        'Dashboard Explanation:\n'
        '• Green bars: Prices increasing\n'
        '• Red bars: Prices decreasing\n'
        '• Blue bars: Trading volume\n'
        '• Price ranges show daily high/low with current price'
    )
    plt.figtext(0.02, 0.96, explanation_text, fontsize=9, style='italic', 
               bbox=dict(boxstyle="round,pad=0.5", facecolor="#ecf0f1", alpha=0.8))
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.93, bottom=0.08)
    plt.show()
    
    # Console output
    print("LIVE CRYPTOCURRENCY DATA:")
    print("=" * 65)
    for item in data:
        trend_indicator = "(UP)" if item['is_positive'] else "(DOWN)"
        print(f"{item['symbol']}: ${item['price']:>8,.2f} {trend_indicator:5} "
              f"Change: {item['price_change_percent']:+.2f}% | "
              f"Volume: {item['volume']:,.0f}")
    print("=" * 65)
    print(f"Market Summary: {positive_count} assets positive, {negative_count} assets negative")

## Dashboard Runner

In [0]:
def run_dashboard(update_interval=15):
    """Run the real-time dashboard"""
    print("Starting Real-Time Cryptocurrency Dashboard")
    print("=" * 50)
    print("Features:")
    print("- Live price updates every 15 seconds")
    print("- Professional visualizations")
    print("- Market sentiment analysis")
    print("- Trading volume metrics")
    print("=" * 50)
    print("Press the STOP button to exit the dashboard")
    print("=" * 50)
    
    try:
        update_count = 0
        while True:
            update_count += 1
            current_time = datetime.now().strftime("%H:%M:%S")
            
            print(f"\nUpdate #{update_count} - {current_time}")
            print("Fetching live market data...")
            
            # Fetch new data
            current_data = fetch_binance_data()
            
            if current_data:
                print("Data received successfully")
                print("Updating dashboard...")
                
                # Update dashboard
                create_professional_dashboard(current_data)
                
                print(f"Dashboard updated successfully at {current_time}")
                print(f"Next update in {update_interval} seconds...")
                
            else:
                print("Warning: No data received - please check internet connection")
            
            # Wait for next update
            time.sleep(update_interval)
            
    except KeyboardInterrupt:
        print("\nDashboard stopped by user")
        print("Thank you for using the Cryptocurrency Analytics Dashboard")
    except Exception as e:
        print(f"Error: Dashboard stopped unexpectedly - {e}")

# Start the dashboard
run_dashboard(update_interval=30)